# This notebook was created to calculate the number of isolates that have 100x coverage in $\ge 99\%$ of sites given a certain locus.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import vcf

%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
import ast

import time
import sys
import pickle
from slurmpy import Slurm
import shutil

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio import Phylo

from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
import itertools
import gzip
import scipy
from collections import Counter

import allel # import scikit-allel

################################################################################################################################################################################################################

# [1] Write script that takes in the path to a full zipped VCF files, extracts just the *POS* and *VD* fields and stores this data as a Series

################################################################################################################################################################################################################

In [6]:
tag = 'SAMN08732238'

In [5]:
import allel # import scikit-allel
import pandas as pd

In [8]:
# genomic data directory
rolling_DB_dir = '/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/'

# path to full zipped VCF
zipped_VCF = rolling_DB_dir + tag + '/pilon/' + tag + '_full.vcf.gz'

# load in the Position & Valid Depth fields from the VCF file
POS_DP_fields = allel.read_vcf(zipped_VCF, fields=['variants/POS', 'variants/DP'])

# create a series with H37Rv Reference Positions as the Index & Depth as the values
depth_series = pd.Series(POS_DP_fields['variants/DP'], index = POS_DP_fields['variants/POS'])

# pickle series for downstream analyses
valid_depth_VCF_dir = '/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/'
depth_series.to_pickle('{0}{1}_POS_VD.pkl'.format(valid_depth_VCF_dir, tag))

################################################################################################################################################################################################################

# [2] Submit jobs to subset full *VCF* files to just the *Position* and *Valid Depth* and store result as Series

################################################################################################################################################################################################################

#### Load INDEL genotype matrix *Isolate* Annotation Files

In [3]:
#load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')

In [4]:
isolate_annotation_DF.head()

,isolate_ID,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,lineage_call,group
0,SAMN13051687,2,2,1,1,1,i3,NaN,NaN,NaN,NaN,NaN,2.2.1.1.1.i3,2
1,SAMN09100245,4,2,1,2,1,1,i3,2,NaN,NaN,NaN,4.2.1.2.1.1.i3.2,4B
2,SAMN08732238,2,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,2.2.1.1.1,2
3,SAMN07658260,3,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1.1,3
4,SAMN03648003,2,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,2.2.1.1.1,2


#### Filter out isolates that have already been processed

In [5]:
# get the file names of the depth series (isolates that have been processed)
isolate_depth_series_files = os.listdir('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series')

# get the isolate tags for these depth series
isolate_IDs_processed = set([isolate_depth_series_file.split('_')[0] for isolate_depth_series_file in isolate_depth_series_files])

# get a boolean for the isolates that have NOT been processed
isolate_IDs_not_processed_filter = [(isolate_ID not in isolate_IDs_processed) for isolate_ID in isolate_annotation_DF.isolate_ID]

# subset isolate annotation for isolates that need to be processed and re-index
isolate_annotation_DF = isolate_annotation_DF[isolate_IDs_not_processed_filter]
isolate_annotation_DF.reset_index(drop = True, inplace = True)

In [6]:
isolate_annotation_DF.head()

,isolate_ID,lineage_1,lineage_2,lineage_3,lineage_4,lineage_5,lineage_6,lineage_7,lineage_8,lineage_9,lineage_10,lineage_11,lineage_call,group
0,SAMN08214998,4,2,1,2,2,1,1,NaN,NaN,NaN,NaN,4.2.1.2.2.1.1,4B


In [7]:
np.shape(isolate_annotation_DF)

(1, 14)

##### Subset VCF to POSITION & VALID DEPTH fields for isolates that had Full VCF files 

Create BASH scripts to extract VCFs of **N** isolates per job

In [8]:
# specify N isolates per job
N = 600

In [9]:
njobs = int( np.ceil( float( len( isolate_annotation_DF.isolate_ID ) ) / float(N) ) ) #number of jobs required if we split for every 600 isolates
isolate_ID_series = pd.Series(isolate_annotation_DF.isolate_ID)

In [10]:
njobs

1

In [11]:
#genomic data directory
rolling_DB_dir = '/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/'

bash_scripts = []

#split jobs up into tasks of N
for first_isolate_i_for_job in np.arange(0 , (njobs)*N , N):

    last_isolate_i_for_job = first_isolate_i_for_job + N
    
    commands_list = []
    
    for isolate_i in range(first_isolate_i_for_job , last_isolate_i_for_job): 

        try:
            tag = isolate_ID_series[isolate_i]
            commands_list.append( 'python /home/rv76/Farhat_Lab/Python_Scripts/homoplasy_project/mmpR_BDQ_mutant_project/subset_fullVCF_to_POS_and_DEPTH.py {0}'.format(tag) )
            
        except KeyError: #applies only to the last job if it has < N isolates
            continue
        
    bash_scripts.append(commands_list)

##### Submit each job to O2

In [12]:
job_num = 1
for job_i_commands_list in bash_scripts:
    
    #append all commands in a single string to be submitted as a job
    VCF_extract_job = ''
    for command_i in job_i_commands_list:
        VCF_extract_job  = VCF_extract_job  + '\n' + command_i
    
    #directory where you want output + error files
    os.chdir('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/depth_series_jobs/')

    job_name = 'VCF_' + str(job_num)

    s = Slurm(job_name , {'partition':'short' , 'n':'1' , 't':'0-3:00:00' , 'mem':'4G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(VCF_extract_job)

    print job_name
    job_num += 1

VCF_1


submitted: Submitted batch job 25866808


################################################################################################################################################################################################################

# [3] Write script that takes in two coordinates corresponding to a region of H37Rv and calculates the # of isolates that have *X* coverage in $\ge 99\%$ of sites in that region

################################################################################################################################################################################################################

In [5]:
H37Rv_start_coord = 778990
H37Rv_stop_coord = 779487
min_coverage_threshold = 100
directory_name = 'mmpR_mod'

In [6]:
import pandas as pd
import os
import sys
import shutil

#### Create directory for text files

In [7]:
directory_for_txt_files = '/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/prop_isolates_with_min_coverage_for_locus/{0}/'.format(directory_name)

# create directory to store progress txt file & txt file with the number of isolates that have adequate coverate for locus
if os.path.exists(directory_for_txt_files):
    shutil.rmtree(directory_for_txt_files)
    os.makedirs(directory_for_txt_files)
elif not os.path.exists(directory_for_txt_files):
    os.makedirs(directory_for_txt_files)

### *Function* that calculates proportion of a locus that has adequate coverage for an isolate

In [8]:
def get_proportion_of_interval_covered(H37Rv_start, H37Rv_stop, coverage_threshold, isolate_ID):
    '''
    This function takes as INPUT start/stop coordinates for a region of H37Rv, a specific coverage threshold and an isolate tag.
    The OUTPUT is the proportion of the interval for the specific isolate that had coverage >= coverage threshold.
    '''
    # load in the series for the depth across the genome
    isolate_ID_depth_series = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/{0}_POS_VD.pkl'.format(isolate_ID))

    # get the valid depth values for a specific region of H37Rv
    isolate_ID_depth_interval = isolate_ID_depth_series[(isolate_ID_depth_series.index >= H37Rv_start) & (isolate_ID_depth_series.index <= H37Rv_stop)]

    # get the proportion of the interval that has a valid depth >= coverage threshold
    prop_interval_covered = ( float(sum(isolate_ID_depth_interval >= coverage_threshold)) / float(len(isolate_ID_depth_interval)) ) * 100.0
    
    return prop_interval_covered

#### Load *mixed* INDEL Isolate Annotation File

In [9]:
# load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/Genotypes_Mixed_only/genotypes_isolate_annotation.pkl')

### Calculate the # of isolates that have *X* coverage in $\ge 99\%$ of sites in the specific region

In [23]:
# set threshold for adequate coverage for a single isolate (99% of the interval has adequate coverage)
adequate_interval_covered_thresh = 99.0 

# get proportion of isolates with adequate coverage in specified interval
num_isolates_with_adeq_cov = 0

# save isolate IDs for checking if mixed INDELs are present in high coverage isolates
isolates_with_adeq_cov_series = pd.Series()

for isolate_i, isolate_ID in zip(isolate_annotation_DF.index, isolate_annotation_DF.isolate_ID):
    
    if get_proportion_of_interval_covered(H37Rv_start_coord, H37Rv_stop_coord, min_coverage_threshold, isolate_ID) >= adequate_interval_covered_thresh:
        num_isolates_with_adeq_cov += 1 #increment count by 1
        isolates_with_adeq_cov_series.loc[num_isolates_with_adeq_cov] = isolate_ID #store isolate ID in series
        
    if isolate_i % 500 == 0:
        with open('{0}progress_check.txt'.format(directory_for_txt_files), 'a') as myfile:
            myfile.write( str( float(isolate_i)/float(len(isolate_annotation_DF.index)) ) + '\n' )
            
# once we iterate through all 31,428 isolates, store number of isolates with adequate coverage in txt file
file = open('{0}num_isolates_high_coverage.txt'.format(directory_for_txt_files), 'w')
file.write(str(num_isolates_with_adeq_cov))
file.close()

# save the list of Isolates IDs as a pickled series & CSV file
isolates_with_adeq_cov_series.to_pickle('{0}isolates_high_coverage.pkl'.format(directory_for_txt_files))
isolates_with_adeq_cov_series.to_csv('{0}isolates_high_coverage.csv'.format(directory_for_txt_files))

################################################################################################################################################################################################################

# [4] Submit script above as a job to calculate the # of isolates that have 100x coverage in $\ge 99\%$ of sites in the *mmpL5-mmpS5-mmpR* operon

################################################################################################################################################################################################################

- **mmpL5** H37Rv start: 775586, H37Rv end: 778480
- **mmpS5** H37Rv start: 778477, H37Rv end: 778905
- **mmpR** H37Rv start: 778990, H37Rv end: 779487
- **eis** H37Rv start: 2714124, H37Rv end: 	2715332
- **whiB7** H37Rv start: 3568401, H37Rv end: 3568679
- **ahpC** H37Rv start: 2726193, H37Rv end: 2726780

In [28]:
H37Rv_start_coord = '778990'
H37Rv_stop_coord = '779487'
min_coverage_threshold = '100'
directory_name = 'mmpR'

In [29]:
#create the job
num_isolates_adequate_depth_cov_job = 'python /home/rv76/Farhat_Lab/Python_Scripts/homoplasy_project/mmpR_BDQ_mutant_project/prop_isolates_with_adequate_depth_for_locus.py {0} {1} {2} {3}'.format(H37Rv_start_coord, H37Rv_stop_coord, min_coverage_threshold, directory_name)

#directory where you want output + error files
os.chdir('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/prop_isolates_with_min_coverage_for_locus/num_isolates_adequate_coverage_jobs')

job_name = directory_name

s = Slurm(job_name , {'partition':'medium' , 'n':'1' , 't':'1-12:00:00' , 'mem':'96G' , 'mail-type':'ALL' , 'mail-user':'roger_vargas@g.harvard.edu'})

#submits the job
job_id = s.run(num_isolates_adequate_depth_cov_job)

print job_name

mmpR


submitted: Submitted batch job 29568014


################################################################################################################################################################################################################

# [5] Number of isolates with 100x coverage in that locus in *mmpR*, *mmpS5* and *mmpL5*

################################################################################################################################################################################################################

In [35]:
mmpS5_isolates = list(pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/prop_isolates_with_min_coverage_for_locus/mmpS5/isolates_high_coverage.pkl'))
mmpL5_isolates = list(pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/prop_isolates_with_min_coverage_for_locus/mmpL5/isolates_high_coverage.pkl'))
mmpR_isolates = list(pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/rolling_DB_scrape_indels/depth_series/prop_isolates_with_min_coverage_for_locus/mmpR/isolates_high_coverage.pkl'))

**mmpR**

In [46]:
len(mmpR_isolates)

7435

**mmpS5**

In [51]:
len(mmpS5_isolates)

8949

**mmpL5**

In [53]:
len(mmpL5_isolates)

6217